# Develop a DCGAN model for CIFAR-10 small object Photograph Dataset, for implement this model - create a generator and discriminator model and then generate the new set of fake images to ensure the GAN outcome.

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar10
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Load and preprocess the CIFAR-10 dataset
(train_images, _), (_, _) = cifar10.load_data()
train_images = (train_images - 127.5) / 127.5  # Normalize to [-1, 1]

In [3]:
# Generator model
def build_generator(latent_dim):
    model = models.Sequential()
    model.add(layers.Dense(4 * 4 * 256, input_dim=latent_dim))
    model.add(layers.Reshape((4, 4, 256)))
    model.add(layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Conv2DTranspose(64, kernel_size=4, strides=2, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Conv2DTranspose(3, kernel_size=4, strides=2, padding='same', activation='tanh'))
    return model

In [4]:
# Discriminator model
def build_discriminator(img_shape):
    model = models.Sequential()
    model.add(layers.Conv2D(64, kernel_size=3, strides=2, padding='same', input_shape=img_shape))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.4))
    model.add(layers.Conv2D(128, kernel_size=3, strides=2, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.4))
    model.add(layers.Conv2D(256, kernel_size=3, strides=2, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.4))
    model.add(layers.Flatten())
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

In [5]:
# Combine generator and discriminator into a GAN model
def build_gan(generator, discriminator):
    discriminator.trainable = False
    model = models.Sequential()
    model.add(generator)
    model.add(discriminator)
    return model

In [6]:
# Set parameters
latent_dim = 100
img_shape = (32, 32, 3)

In [7]:
# Build and compile the discriminator
discriminator = build_discriminator(img_shape)
discriminator.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(0.0002, 0.5), metrics=['accuracy'])

In [8]:
# Build the generator
generator = build_generator(latent_dim)

In [9]:
# Build and compile the GAN model
discriminator.trainable = False
gan = build_gan(generator, discriminator)
gan.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(0.0002, 0.5))

In [10]:
# Training the GAN
epochs = 30000
batch_size = 64
half_batch = batch_size // 2

In [11]:
for epoch in range(epochs):
    # Select a random half batch of real images
    idx = np.random.randint(0, train_images.shape[0], half_batch)
    real_imgs = train_images[idx]

In [12]:
# Generate a half batch of fake images
noise = np.random.normal(0, 1, (half_batch, latent_dim))
fake_imgs = generator.predict(noise)

1/1 [==============================] - 1s 1s/step


In [13]:
# Train the discriminator on real and fake images
d_loss_real = discriminator.train_on_batch(real_imgs, np.ones((half_batch, 1)))
d_loss_fake = discriminator.train_on_batch(fake_imgs, np.zeros((half_batch, 1)))
d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

In [14]:
# Train the generator to fool the discriminator
noise = np.random.normal(0, 1, (batch_size, latent_dim))
valid_labels = np.ones((batch_size, 1))
g_loss = gan.train_on_batch(noise, valid_labels)

In [15]:
# Print progress and save generated images at intervals
if epoch % 100 == 0:
    print(f"Epoch {epoch}, [D loss: {d_loss[0]} | D accuracy: {100 * d_loss[1]}] [G loss: {g_loss}]")

In [16]:
# Save generated images
if epoch % 1000 == 0:
    save_path = f"generated_images/gen_img_epoch_{epoch}.png"
    gen_imgs = generator.predict(np.random.normal(0, 1, (16, latent_dim)))
    gen_imgs = 0.5 * gen_imgs + 0.5  # Rescale to [0, 1]
    fig, axs = plt.subplots(4, 4, figsize=(4, 4), sharex=True, sharey=True)
    cnt = 0
    for i in range(4):
        for j in range(4):
            axs[i, j].imshow(gen_imgs[cnt, :, :, :])
            axs[i, j].axis('off')
            cnt += 1
    plt.savefig(save_path)
    plt.close()